# Day 02 — Final scripts + Gradio UI

**Goal:** collect the finished train/infer scripts into a repeatable workflow, add a lightweight UI, and document VRAM profiles + troubleshooting.

**Artifacts in repo:**
- Final scripts: `scripts/finals/train.sh`, `scripts/finals/infer.sh`
- Inference UI: `scripts/ui/app.py`  (exposes Day-6 sampler via Gradio)
- Make targets: `make final-train-a`, `make final-train-b`, `make final-infer`, `make ui`

This notebook summarizes how to run **training (Stage-A/B)**, **CLI inference**, and **UI inference**, plus fixes for common UI dependency hiccups (e.g., “No API found” / schema crash).


## 0) Repo map (relevant to Day 02)

```
scripts/
  finals/
    train.sh        # Stage A/B driver (reads VRAM envs)
    infer.sh        # CLI inference (Day-6 sampler)
  ui/
    app.py          # Gradio app (mp4+wav)
configs/
  day05_train.yaml  # Stage-A config
  day07_trainB.yaml # Stage-B config
  day06_infer.yaml  # Day-6 sampler config used by CLI/UI
  ui_infer.yaml     # UI defaults (paths)
```


## 1) VRAM profiles (export **before** running)

These control time-token truncation and attention chunking/downsampling. They’re honored by both Make targets and the shell scripts.


In [ ]:
%%bash
## Choose ONE profile and run in your shell session

echo "# Max-Quality (>=80GB)"
cat <<'EOF'
export JOINTDIT_MAX_T=12
export JOINTDIT_Q_CHUNK_V=128
export JOINTDIT_Q_CHUNK_A=0
export JOINTDIT_KV_DOWNSAMPLE=4
export PYTORCH_CUDA_ALLOC_CONF="max_split_size_mb:128"
EOF

echo
echo "# Balanced (~48GB)"
cat <<'EOF'
export JOINTDIT_MAX_T=12
export JOINTDIT_Q_CHUNK_V=128
export JOINTDIT_Q_CHUNK_A=0
export JOINTDIT_KV_DOWNSAMPLE=4
export PYTORCH_CUDA_ALLOC_CONF="max_split_size_mb:128"
EOF

echo
echo "# Conservative (<=24GB)"
cat <<'EOF'
export JOINTDIT_MAX_T=6
export JOINTDIT_Q_CHUNK_V=64
export JOINTDIT_Q_CHUNK_A=0
export JOINTDIT_KV_DOWNSAMPLE=8
export PYTORCH_CUDA_ALLOC_CONF="max_split_size_mb:128"
EOF


## 2) Training — Stage-A / Stage-B

Both paths accept `--max-steps`, `--ckpt-suffix`, and `--log-suffix`.


In [ ]:
%%bash
# Stage A (Make or direct)
make final-train-a
# Or:
PYTHONPATH=. scripts/finals/train.sh \
  --stage A --cfg configs/day05_train.yaml \
  --max-steps 25 --ckpt-suffix finalA --log-suffix finalA

# Stage B (Make or direct)
make final-train-b
# Or:
PYTHONPATH=. scripts/finals/train.sh \
  --stage B --cfg configs/day07_trainB.yaml \
  --max-steps 1000 --ckpt-suffix finalB --log-suffix finalB


## 3) Inference — CLI (Day-6 sampler)

Outputs MP4 and WAV using the VAEs + sampler settings from Day-6.


In [ ]:
%%bash
make final-infer
# Or:
PYTHONPATH=. scripts/finals/infer.sh \
  --cfg configs/day06_infer.yaml \
  --ckpt checkpoints/day07_stage_b_finalB/ckpt_step_001000.pt \
  --steps 30 --seed 0


## 4) Inference — Gradio UI

Start the app and open the URL (default: http://127.0.0.1:7860). Fill paths, pick VRAM profile, set steps/CFG, then **Run**.

**Known dependency gotchas** and fixes are below.


In [ ]:
%%bash
make ui


### 4.1) UI Troubleshooting (dependency pins)

**If you see:** “No API found” or a JSON-schema crash from `gradio_client.utils` → you’re hitting a FastAPI/Starlette/Gradio schema mismatch. Two proven pin sets:

**Option A (stable with current repo):**
```bash
pip install -U "altair<6" "numpy==1.26.4"
pip install --no-deps --force-reinstall "gradio==4.19.2" "gradio_client==0.10.1"
pip install "fastapi<0.112" "starlette<0.37" "uvicorn<0.30"
```

**Option B (newer gradio):**
```bash
pip install -U "gradio==4.44.0" "gradio_client==1.4.2"
pip check
```

The app also includes a small **schema safety patch** that bypasses the problematic API schema route.


## 5) Audio notes

- Ensure `torchaudio` is installed for Griffin-Lim inversion:
  ```bash
  pip install torchaudio
  ```
- Inversion uses meta values: `n_mels`, `hop_length`, `win_length`, `fmin`, `fmax`. Silent WAVs usually mean a mismatch with cached meta.


## 6) Outputs & checkpoints

- UI writes to `outputs/ui/`
- CLI Day-6 writes to `outputs/day06/`
- Checkpoints: `checkpoints/day05_stage_a_*` and `checkpoints/day07_stage_b_*`

**Tip:** the Makefile sets `FORCE_KEEP_USER_ENVS=1` so your exported VRAM envs override defaults.
